In [ ]:
%%script bash
mkdir scrapers/
mkdir scrapers/amazon
cd scrapers/amazon
python -m pip install virtualenv
virtualenv env
. env/bin/activate
python -m pip install scrapy
python -m pip install numpy
python -m pip install pandas
python -m pip install requests
python -m pip install scraperapi-sdk
scrapy startproject amazon_scraper